# BookSync
Менеджер книг
***
Функции:
* **Пользователи** могуть:
    * лайкать книги (добавлять в избранное)
    * писать отзывы и ставить оценки
    * искать книги по названию, и т.п., группировать по жанру, сортировать по рейтингу
* **Книги**
    * отображают средний рейтинг

In [26]:
import pymongo  
from pymongo import errors, DESCENDING, ASCENDING

from bson import ObjectId
from pprint import pprint
import bcrypt

In [3]:
client = pymongo.MongoClient('localhost', 27017)

In [4]:
# проверить соединение
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [5]:
booksync = client.booksync

## Добавление книг

In [6]:
book_validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['title', 'author', 'description', 'genres', 'year'],
        'properties': {
            'title': {
                'bsonType': 'string',
            },
            'author': {
                'bsonType': 'string',
            },
            'description': {
                'bsonType': 'string',
            },
            'year': {
                'bsonType': 'int',
            },
            'avg_rating': {
                'bsonType': 'double',
            },
            'genres': {
                'bsonType': 'array',
                'items': {
                    'bsonType': 'string'
                },
                'minItems': 1,
                'uniqueItems': True
            },
            'users_liked': {
                'bsonType': 'array',
                'items': {
                    'bsonType': 'objectId'
                },
                'minItems': 1,
                'uniqueItems': True
            },
            'reviews': {
                'bsonType': 'array',
                'items': {
                    'bsonType': 'objectId',
                        }
                    }
                }
            }
        }


In [7]:
# booksync.drop_collection('books')

In [8]:
booksync.create_collection('books', validator=book_validator)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'booksync'), 'books')

In [9]:
booksync.books.create_index([("title", 1), ("author", 1)], unique=True)


'title_1_author_1'

In [10]:
def add_book(title, author, description, genres, year):

    if not all([title, author, description, genres, year]):
        print("Ошибка: не все обязательные поля заполнены.")
        return None

    book = {
        "title": title,
        "author": author,
        "description": description,
        "genres": genres,
        "year": year,
        "reviews": []
    }
    
    try:
        result = booksync.books.insert_one(book)
        print(f"✅ Книга добавлена с ID: {result.inserted_id}")
        return result.inserted_id
    except errors.DuplicateKeyError as e:
        print(f" ❌Книга ужe существует с ID: {booksync.books.find({'title': title, 'author': author})[0]['_id']}")



In [11]:
# Список книг для добавления
books = [
    {
        "title": "Мастер и Маргарита",
        "author": "Михаил Булгаков",
        "description": "Роман о дьяволе, любви и свободе, действие которого разворачивается в Москве и древнем Иерусалиме.",
        "genres": ["Классика", "Мистика", "Фантастика"],
        "year": 1967
    },
    {
        "title": "Преступление и наказание",
        "author": "Фёдор Достоевский",
        "description": "Философский роман о нравственных терзаниях студента Раскольникова, совершившего преступление.",
        "genres": ["Классика", "Психологическая драма"],
        "year": 1866
    },
    {
        "title": "Война и мир",
        "author": "Лев Толстой",
        "description": "Монументальный роман о войне 1812 года, судьбах дворянских семей и философских размышлениях о жизни.",
        "genres": ["Классика", "Исторический роман"],
        "year": 1869
    },
    {
        "title": "Пикник на обочине",
        "author": "Аркадий и Борис Стругацкие",
        "description": "Научно-фантастический роман о загадочных Зонах и сталкерах, исследующих их тайны.",
        "genres": ["Фантастика", "Научная фантастика"],
        "year": 1972
    },
    {
        "title": "1984",
        "author": "Джордж Оруэлл",
        "description": "Антиутопия о тоталитарном обществе, где 'Большой Брат' следит за всеми.",
        "genres": ["Дистопия", "Политическая фантастика"],
        "year": 1949
    }
]

In [12]:
# Добавляем книги в базу
for book in books:
    add_book(**book)


✅ Книга добавлена с ID: 67e6e56531bb3cdab1d6128e
✅ Книга добавлена с ID: 67e6e56531bb3cdab1d6128f
✅ Книга добавлена с ID: 67e6e56531bb3cdab1d61290
✅ Книга добавлена с ID: 67e6e56531bb3cdab1d61291
✅ Книга добавлена с ID: 67e6e56531bb3cdab1d61292


In [13]:
# Добавляем книги в базу
for book in books:
    add_book(**book)


 ❌Книга ужe существует с ID: 67e6e56531bb3cdab1d6128e
 ❌Книга ужe существует с ID: 67e6e56531bb3cdab1d6128f
 ❌Книга ужe существует с ID: 67e6e56531bb3cdab1d61290
 ❌Книга ужe существует с ID: 67e6e56531bb3cdab1d61291
 ❌Книга ужe существует с ID: 67e6e56531bb3cdab1d61292


## Пользователи

In [14]:
user_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["username", "password_hash"],
        "properties": {
            "username": {
                "bsonType": "string",
                "description": "Уникальный логин пользователя"
            },
            "password_hash": {
                "bsonType": "string",
                "description": "Хешированный пароль"
            },
            "favorite_books": {
                "bsonType": "array",
                "items": {
                    "bsonType": "objectId",
                }
                },
            "reviews": {
                "bsonType": "array",
                "items": {
                    "bsonType": "objectId",
                }
            }
        
        }
    }
}


In [15]:
# booksync.drop_collection('users')

In [16]:
booksync.create_collection('users', validator=user_validator)
booksync.users.create_index("username", unique=True)

'username_1'

In [17]:
def hash_password(password):
    """Хеширует пароль с использованием bcrypt."""
    salt = bcrypt.gensalt()
    return bcrypt.hashpw(password.encode('utf-8'), salt).decode('utf-8')

def add_user(username, password):
    """
    Добавляет пользователя в базу, если username уникален.
    
    :param username: str – логин (должен быть уникальным)
    :param email: str – email
    :param password: str – пароль (он будет хеширован)
    :param favorite_books: list – список ObjectId любимых книг (опционально)
    :return: ObjectId – ID добавленного пользователя или None, если имя занято
    """
    if not all([username, password]):
        print("❌ Ошибка: все поля должны быть заполнены.")
        return None

    user = {
        "username": username,
        "password_hash": hash_password(password),
    }

    try:
        result = booksync.users.insert_one(user)
        print(f"✅ Пользователь добавлен с ID: {result.inserted_id}")
    except errors.DuplicateKeyError:
        print(f"❌ Ошибка: имя пользователя '{username}' уже занято.")


In [18]:
add_user("reader1", "securepassword123")
add_user("reader1", "securepassword456")  # Повторное имя
add_user("Nastya", "1234")
add_user("Yanina", "101010101")


✅ Пользователь добавлен с ID: 67e6e63b31bb3cdab1d61298
❌ Ошибка: имя пользователя 'reader1' уже занято.
✅ Пользователь добавлен с ID: 67e6e63c31bb3cdab1d6129a
✅ Пользователь добавлен с ID: 67e6e63c31bb3cdab1d6129b


In [19]:
#Добавить книгу в избранное
def like(username, book_title, author):
    user_id = booksync.users.find({'username': username})[0]['_id']
    book_id = booksync.books.find({'title': book_title, 'author': author})[0]['_id']

    # Добавляем ссылку на отзыв в пользователя
    booksync.users.update_one(
        {"_id": ObjectId(user_id)},
        {"$push": {"favorite_books": book_id}}
    )

In [20]:
like('Nastya', 'Мастер и Маргарита', 'Михаил Булгаков')
like('Nastya', 'Преступление и наказание', 'Фёдор Достоевский')
like('Nastya', 'Война и мир', 'Лев Толстой')

## Отзывы
Айди отзыва харинтся у пользователя и у книги

После добавления отзыва обновляется средний рейтинг книги

In [21]:
review_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["user_id", "book_id", "rating"],
        "properties": {
            "user_id": {
                "bsonType": "objectId",
            },
            "book_id": {
                "bsonType": "objectId",
            },
            "text": {
                "bsonType": "string",
            },
            "rating": {
                    'bsonType': 'int',
                    'minimum': 1,
                    'maximum': 5
            }
        }
    }
}


In [22]:
# booksync.drop_collection('reviews')

In [23]:
booksync.create_collection('reviews', validator=review_validator)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'booksync'), 'reviews')

In [24]:
def add_review(username, book_title, author, rating, text=""):
    user_id = booksync.users.find({'username': username})[0]['_id']
    book_id = booksync.books.find({'title': book_title, 'author': author})[0]['_id']

    review = {
        "user_id": ObjectId(user_id),
        "book_id": ObjectId(book_id),
        "rating": rating,
        "text": text
    }

    review_result = booksync.reviews.insert_one(review)
    review_id = review_result.inserted_id

    booksync.users.update_one(
        {"_id": ObjectId(user_id)},
        {"$push": {"reviews": review_id}}
    )

    booksync.books.update_one(
        {"_id": ObjectId(book_id)},
        {"$push": {"reviews": review_id}}
    )

    update_avg_rating(book_id)

    print(f"✅ Отзыв {review_id} добавлен!")
    return review_id

def update_avg_rating(book_id):
    """
    Пересчитывает средний рейтинг книги по всем отзывам.

    :param book_id: ObjectId - ID книги
    """
    book_id = ObjectId(book_id)

    # Получаем все рейтинги для данной книги
    ratings = booksync.reviews.find({"book_id": book_id}, {"rating": 1})
    ratings_list = [r["rating"] for r in ratings]

    if ratings_list:
        avg_rating = sum(ratings_list) / len(ratings_list)
        booksync.books.update_one({"_id": book_id}, {"$set": {"avg_rating": avg_rating}})


In [25]:
add_review('Nastya', 'Мастер и Маргарита', 'Михаил Булгаков', 5, 'Отличная книга! В самое сердце')
add_review('Yanina', 'Мастер и Маргарита', 'Михаил Булгаков', 4, 'Интересно, но немного затянуто')

add_review('reader1', '1984', 'Джордж Оруэлл', 5, 'Очень актуальная книга!')
add_review('Nastya', '1984', 'Джордж Оруэлл', 1, 'Типичная антиутопия, ничего нового')
add_review('Yanina', '1984', 'Джордж Оруэлл', 5)

add_review('Nastya', 'Преступление и наказание', 'Фёдор Достоевский', 5, 'Сложно, но достойно внимания')
add_review('Yanina', 'Преступление и наказание', 'Фёдор Достоевский', 3, 'Глубокий смысл, шикарно!')

add_review('reader1', 'Пикник на обочине', 'Аркадий и Борис Стругацкие', 5)
add_review('Nastya', 'Пикник на обочине', 'Аркадий и Борис Стругацкие', 2)
add_review('Yanina', 'Пикник на обочине', 'Аркадий и Борис Стругацкие', 4)

add_review('reader1', 'Война и мир', 'Лев Толстой', 5, 'Книга моего дества! Сейчас взглянул с новой стороны')
add_review('Nastya', 'Война и мир', 'Лев Толстой', 1)
add_review('Yanina', 'Война и мир', 'Лев Толстой', 3)

✅ Отзыв 67e6e6de31bb3cdab1d6129c добавлен!
✅ Отзыв 67e6e6de31bb3cdab1d6129d добавлен!
✅ Отзыв 67e6e6de31bb3cdab1d6129e добавлен!
✅ Отзыв 67e6e6de31bb3cdab1d6129f добавлен!
✅ Отзыв 67e6e6de31bb3cdab1d612a0 добавлен!
✅ Отзыв 67e6e6de31bb3cdab1d612a1 добавлен!
✅ Отзыв 67e6e6de31bb3cdab1d612a2 добавлен!
✅ Отзыв 67e6e6de31bb3cdab1d612a3 добавлен!
✅ Отзыв 67e6e6de31bb3cdab1d612a4 добавлен!
✅ Отзыв 67e6e6de31bb3cdab1d612a5 добавлен!
✅ Отзыв 67e6e6df31bb3cdab1d612a6 добавлен!
✅ Отзыв 67e6e6df31bb3cdab1d612a7 добавлен!
✅ Отзыв 67e6e6df31bb3cdab1d612a8 добавлен!


ObjectId('67e6e6df31bb3cdab1d612a8')

# Функции
1. Найти книгу по названию / жанру / автору / году
2. Отсортировать книги по рейтингу
3. Сгруппировать книги по жанру
4. Посмотреть отзывы на книгу
5. Посмотреть свои лайкнутые книги

In [28]:
# Поиск
def find_books(title=None, author=None, genre=None, year=None):
    """
    Ищет книги по названию, автору, жанру или году.

    :param title: str - Название книги (опционально)
    :param author: str - Автор книги (опционально)
    :param genre: str - Жанр (опционально)
    :param year: int - Год выпуска (опционально)
    :return: list - Найденные книги
    """
    query = {}

    if title:
        query["title"] = {"$regex": title, "$options": "i"}  # Регистронезависимый поиск
    if author:
        query["author"] = {"$regex": author, "$options": "i"}
    if genre:
        query["genres"] = genre  # Жанры хранятся как массив
    if year:
        query["year"] = year

    books = list(booksync.books.find(query, {"_id": 0}))
    return books

In [30]:
pprint(find_books(title="1984"))
pprint(find_books(author="Булгаков", genre="Классика"))
pprint(find_books(year=2023))

[{'author': 'Джордж Оруэлл',
  'avg_rating': 3.6666666666666665,
  'description': "Антиутопия о тоталитарном обществе, где 'Большой Брат' "
                 'следит за всеми.',
  'genres': ['Дистопия', 'Политическая фантастика'],
  'reviews': [ObjectId('67e6e6de31bb3cdab1d6129e'),
              ObjectId('67e6e6de31bb3cdab1d6129f'),
              ObjectId('67e6e6de31bb3cdab1d612a0')],
  'title': '1984',
  'year': 1949}]
[{'author': 'Михаил Булгаков',
  'avg_rating': 4.5,
  'description': 'Роман о дьяволе, любви и свободе, действие которого '
                 'разворачивается в Москве и древнем Иерусалиме.',
  'genres': ['Классика', 'Мистика', 'Фантастика'],
  'reviews': [ObjectId('67e6e6de31bb3cdab1d6129c'),
              ObjectId('67e6e6de31bb3cdab1d6129d')],
  'title': 'Мастер и Маргарита',
  'year': 1967}]
[]


In [32]:
# Сортировка книг по рейтингу
def get_top_books(limit=10, ascending=False):
    """
    Получает топ книг по среднему рейтингу.

    :param limit: int - Количество книг (по умолчанию 10)
    :param ascending: bool - False (по убыванию), True (по возрастанию)
    :return: list - Отсортированные книги
    """
    order = ASCENDING if ascending else DESCENDING
    books = list(booksync.books.find({}, {"_id": 0}).sort("avg_rating", order).limit(limit))
    return books


In [34]:
pprint(get_top_books(limit=3))


[{'author': 'Михаил Булгаков',
  'avg_rating': 4.5,
  'description': 'Роман о дьяволе, любви и свободе, действие которого '
                 'разворачивается в Москве и древнем Иерусалиме.',
  'genres': ['Классика', 'Мистика', 'Фантастика'],
  'reviews': [ObjectId('67e6e6de31bb3cdab1d6129c'),
              ObjectId('67e6e6de31bb3cdab1d6129d')],
  'title': 'Мастер и Маргарита',
  'year': 1967},
 {'author': 'Фёдор Достоевский',
  'avg_rating': 4.0,
  'description': 'Философский роман о нравственных терзаниях студента '
                 'Раскольникова, совершившего преступление.',
  'genres': ['Классика', 'Психологическая драма'],
  'reviews': [ObjectId('67e6e6de31bb3cdab1d612a1'),
              ObjectId('67e6e6de31bb3cdab1d612a2')],
  'title': 'Преступление и наказание',
  'year': 1866},
 {'author': 'Аркадий и Борис Стругацкие',
  'avg_rating': 3.6666666666666665,
  'description': 'Научно-фантастический роман о загадочных Зонах и сталкерах, '
                 'исследующих их тайны.',
  '

In [35]:
pprint(get_top_books(limit=1, ascending=True))


[{'author': 'Лев Толстой',
  'avg_rating': 3.0,
  'description': 'Монументальный роман о войне 1812 года, судьбах дворянских '
                 'семей и философских размышлениях о жизни.',
  'genres': ['Классика', 'Исторический роман'],
  'reviews': [ObjectId('67e6e6df31bb3cdab1d612a6'),
              ObjectId('67e6e6df31bb3cdab1d612a7'),
              ObjectId('67e6e6df31bb3cdab1d612a8')],
  'title': 'Война и мир',
  'year': 1869}]


In [38]:
# Группировка книг по жанрам
def group_books_by_genre():
    """
    Группирует книги по жанрам.

    :return: dict - Жанры с книгами
    """
    pipeline = [
        {"$unwind": "$genres"},
        {"$group": {"_id": "$genres", "books": {"$push": "$title"}}}
    ]
    grouped = booksync.books.aggregate(pipeline)

    return {genre["_id"]: genre["books"] for genre in grouped}


In [39]:
pprint(group_books_by_genre())


{'Дистопия': ['1984'],
 'Исторический роман': ['Война и мир'],
 'Классика': ['Мастер и Маргарита', 'Преступление и наказание', 'Война и мир'],
 'Мистика': ['Мастер и Маргарита'],
 'Научная фантастика': ['Пикник на обочине'],
 'Политическая фантастика': ['1984'],
 'Психологическая драма': ['Преступление и наказание'],
 'Фантастика': ['Мастер и Маргарита', 'Пикник на обочине']}


In [50]:
# Получить отзывы на книгу
def get_reviews_for_book(title, author):
    """
    Возвращает отзывы на книгу.

    :param title: str - Название книги
    :param author: str - Автор книги
    :return: list - Список отзывов
    """
    book = booksync.books.find_one({"title": title, "author": author})
    if not book:
        print(f"❌ Ошибка: книга '{title}' ({author}) не найдена!")
        return []

    reviews = list(booksync.reviews.find({"book_id": book["_id"]}, {"_id": 0, "user_id": 1, "rating": 1, "text": 1}))
        # Добавляем имя пользователя к каждому отзыву
    for review in reviews:
        user = booksync.users.find_one({"_id": review["user_id"]}, {"username": 1})
        review["username"] = user["username"] if user else "Неизвестный пользователь"

    return reviews


In [53]:
pprint(get_reviews_for_book("Мастер и Маргарита", "Михаил Булгаков"))

[{'rating': 5,
  'text': 'Отличная книга! В самое сердце',
  'user_id': ObjectId('67e6e63c31bb3cdab1d6129a'),
  'username': 'Nastya'},
 {'rating': 4,
  'text': 'Интересно, но немного затянуто',
  'user_id': ObjectId('67e6e63c31bb3cdab1d6129b'),
  'username': 'Yanina'}]


In [54]:
# Посмотреть лайкнутые книги пользователя
def get_liked_books(username):
    """
    Возвращает список книг, которые лайкнул пользователь.

    :param username: str - Имя пользователя
    :return: list - Список книг
    """
    user = booksync.users.find_one({"username": username})
    if not user:
        print(f"❌ Ошибка: пользователь '{username}' не найден!")
        return []

    liked_books_ids = booksync.users.find({"_id": user["_id"]})[0]['favorite_books']
    liked_books = [booksync.books.find({"_id": id})[0]['title'] for id in liked_books_ids]
    return liked_books


In [55]:
print(get_liked_books("Nastya"))


['Мастер и Маргарита', 'Преступление и наказание', 'Война и мир']
